In [ ]:
%load_ext autoreload

In [ ]:
%autoreload

import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.linalg
import seaborn as sns
import torch
from tqdm.notebook import tqdm
from utils import latexify

torch.set_default_tensor_type(torch.DoubleTensor)

In [ ]:
latexify(6, 4)

In [ ]:
data = pd.read_csv('data/crsp.csv')
ETFS = ['AGG', 'VTI', 'VNQ', 'XLF', 'XLV', 'XLY', 'XLP', 'XLU', 'XLI', 'XLE', 'IBB', 'ITA']
rets = []

for etf in ETFS:
    rets.append(1 + np.array(data[data['TICKER'] == etf]['RET'], dtype=np.float64)[-144:])
    
etf_ret = np.stack(rets, axis=1)

In [ ]:
N_ASSETS = len(ETFS)
HZN = 24
VAL = 2443
TEST = VAL * 3
KAPPA = np.full(N_ASSETS, 0.001)
SHORT = np.full(N_ASSETS, 0.001)
GAMMA = 15.0

M1 = torch.tensor(2.0)
M2 = torch.tensor(1.00)
len(ETFS)

In [ ]:
def ret_and_cov(returns):
    N = returns.shape[1]
    returnslog = np.log(returns)
    mu_returnslog = np.mean(returnslog, axis=0)
    cov_returnslog = np.cov(returnslog.T) + 1e-10 * np.eye(N)

    mu = np.exp(mu_returnslog + .5 * np.diag(cov_returnslog))
    cov = np.diag(mu)@(np.exp(cov_returnslog)-np.ones((N,N)))@np.diag(mu)
    return mu, cov

MU, COV = ret_and_cov(etf_ret)
COV_SQRT = scipy.linalg.sqrtm(COV)

mulog = torch.tensor(np.log(etf_ret)).mean(axis=0)
covlog = torch.tensor(np.cov(etf_ret.T))

In [ ]:
def init_holdings(cov_sqrt, mu, gamma):
    N = N_ASSETS
    ht = cp.Variable(N)   
    
                                                                                                                                                        
    risk = gamma * cp.sum_squares(cov_sqrt @ ht)
    returns = mu.T @ ht
    objective = returns - risk - SHORT.T @ cp.neg(ht)
                                                                                
    constraints = [                                                             
        cp.sum(ht) == 1,
    ]                                                                            
    problem = cp.Problem(cp.Maximize(objective), constraints)
    problem.solve()
    return ht.value

H0 = torch.tensor(init_holdings(COV_SQRT, MU, GAMMA))

In [ ]:
plt.imshow(COV)
plt.colorbar()
plt.xticks(np.arange(N_ASSETS), ETFS, rotation=90)
plt.yticks(np.arange(N_ASSETS), ETFS)
plt.show()

In [ ]:
def plot_mean_std(mu, cov):
    ax = plt.gca()
    eb = plt.errorbar(np.arange(N_ASSETS), mu, yerr=np.sqrt(np.diag(cov)))
    eb[-1][0].set_linestyle('--')
    ax.axhline(1.0, linestyle='--', color='gray')
    plt.xticks(np.arange(N_ASSETS), ETFS, rotation=90)
    
plot_mean_std(MU, COV)
plt.show()

In [ ]:
kappa_tch = torch.from_numpy(KAPPA)
logreturn1p_dist = torch.distributions.MultivariateNormal(
    mulog, covlog)


def utility_fn(x, m1=M1, m2=M2):
    return torch.min(m1*(x - 1), m2*(x - 1))                                                  


def stage_cost(r):                                                       
    return -utility_fn(r)


def simulate(ht, ut):
    ret = torch.exp(logreturn1p_dist.sample())
    ht = ret * (ht + ut)
    return ht, ret


def rollout(policy, h0=H0, seed=None):
    if seed is not None:
        torch.manual_seed(seed)
    ht = h0                                                                    
    h_seq = [ht]                                                               
    u_seq = []    
    rets = []
    cost = 0.0
    for t in range(HZN):                                                    
        ut = policy(ht)
        htp1, _ = simulate(ht, ut)
        ret = htp1.sum() / ht.sum()
        cost += stage_cost(ret).mean()
        ht = htp1
        h_seq.append(ht)                                                        
        u_seq.append(ut)
        rets.append(ret)
    cost = cost / HZN
    return cost, h_seq, u_seq, rets


def monte_carlo(policy, h0=H0, trials=10, seed=None, verbose=True):
    if seed is not None:
        torch.manual_seed(seed)
    results = []
    hN = []
    
    if verbose:
        wrapper = lambda x: tqdm(x)
    else:
        wrapper = lambda x: x
    
    for i in wrapper(range(trials)):
        cost, h_seq, _, rets = rollout(policy, h0)
        results.append(cost)
        hN.append(h_seq[-1])
    return torch.stack(results), hN, rets

In [ ]:
ax = plt.gca()
plt.plot(np.linspace(0.9, 1.1, 1000), [utility_fn(x) for x in np.linspace(.9, 1.1, 1000)])
ax.axvline(1, color='gray', linestyle='--')
plt.show()

In [ ]:
def plot_states(policy, ax=None, h0=H0, seed=TEST, style={},
                legend=False, ylabel=True, xlabel=True):
    if ax is None:
        ax = plt.gca()
    colors = sns.color_palette("hls", N_ASSETS)
    _, states, _, _ = rollout(policy, h0=h0, seed=seed)
    normalized = [s / s.sum() for s in states]
    states_stacked = torch.stack(normalized)
    states_np = states_stacked.detach().numpy()
    for i, etf in enumerate(ETFS):
        holdings = states_np[:, i]
        if etf in style:
            ax.plot(holdings, label=etf, linestyle=style[etf], color=colors[i])
        else:
            ax.plot(holdings, label=etf, color=colors[i])
    if xlabel:
        ax.set_xlabel("month")
    if ylabel:
        ax.set_ylabel("holdings")
    ax.set_xticks(np.arange(HZN, step=4))
    ax.set_xlim((0, HZN))
    ax.set_ylim((-.75, .95))
    if legend:
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=6)
    plt.tight_layout()
    
    
def plot_controls(policy, ax=None, h0=H0, seed=TEST, style={},
                  legend=False, ylabel=True, xlabel=True):
    if ax is None:
        ax = plt.gca()
    _, states, controls, _ = rollout(policy, h0=h0, seed=seed)
    colors = sns.color_palette("hls", N_ASSETS)
    normalized = [c / s.sum() for s, c in zip(states, controls)]
    stacked = torch.stack(normalized)
    controls_np = stacked.detach().numpy()
    for i, etf in enumerate(ETFS):
        ctrl = controls_np[:, i]
        if etf in style:
            ax.plot(ctrl, label=etf, linestyle=style[etf], color=colors[i])
        else:
            ax.plot(ctrl, label=etf, color=colors[i])
    ax.set_xticks(np.arange(HZN, step=4))
    ax.set_xlim((0, HZN))
    if xlabel:
        ax.set_xlabel('month')
    if ylabel:
        ax.set_ylabel('trades')
    if legend:
        ax.legend()
    plt.tight_layout()

In [ ]:
def construct_layer():
    N = N_ASSETS
    ht = cp.Parameter(N)   
    cov_sqrt = cp.Parameter((N, N))
    mu = cp.Parameter(N)
    parameters = [ht, cov_sqrt, mu]                     
                                                                                
    ut = cp.Variable(N)                                                         
    htp = cp.Variable(N)             
                                                                                
    risk = cp.sum_squares(cov_sqrt @ htp)
    returns = mu.T @ htp
    objective = returns - risk
                                                                                
    transaction_cost = KAPPA.T @ cp.abs(ut)
    shorting_cost = SHORT.T @ cp.neg(htp)
    constraints = [                                                             
        cp.sum(ut) + transaction_cost + shorting_cost <= 0,                                   
        htp == ht + ut,
    ]                                                                            
    problem = cp.Problem(cp.Maximize(objective), constraints)
    return CvxpyLayer(problem, parameters, [ut])   

In [ ]:
class Tuner(object):                                                       
    def __init__(self, mu=MU, gamma=GAMMA, cov_sqrt=COV_SQRT):            
        self.layer = construct_layer()  
        
        self.gamma_sqrt = torch.tensor(np.sqrt(gamma), requires_grad=True)
        self.cov_sqrt = torch.tensor(cov_sqrt, requires_grad=True)
        self.mu = torch.tensor(mu, requires_grad=True)
        self.variables = [self.gamma_sqrt, self.cov_sqrt, self.mu]
        self.solver_args= {'acceleration_lookback': 0, 'max_iters': 10000}      

    def __call__(self, ht):
        norm_ht = ht / ht.sum()
        norm_ut = self.layer(
            norm_ht, self.gamma_sqrt * self.cov_sqrt, self.mu,
            solver_args=self.solver_args)[0]
        return norm_ut * ht.sum()

    def project(self):
        self.gamma_sqrt.data = torch.max(self.gamma_sqrt, torch.tensor(0.0))
        
        
untuned = Tuner(gamma=GAMMA, cov_sqrt=COV_SQRT)

In [ ]:
STYLE = {
    'AGG': '--',
    'ITA': ':'
}

In [ ]:
def train(policy, epochs, batch_size=10, lr=1e-2, print_every=50):    
    opt = torch.optim.SGD(policy.variables, lr=lr)                             
    print("Training for {} epochs (batch size={}, lr={})".format(epochs, batch_size, lr))
    test_utils = []
    
    with torch.no_grad():
        test_costs, _, _ = monte_carlo(policy, trials=batch_size, seed=TEST, verbose=False)
        test_utils.append(-test_costs.mean())
        print('start: test utility ({})'.format(test_utils[-1]))
        
    for epoch in tqdm(range(epochs)):  
        
        if (epoch + 1) % 100 == 0:
            lr = lr / 2
            opt = torch.optim.SGD(policy.variables, lr=lr)
            
        with torch.no_grad():
            h0 = init_holdings(policy.cov_sqrt.detach().numpy(), policy.mu.detach().numpy(),
                               policy.gamma_sqrt.detach().item()**2)
            h0 = torch.tensor(h0)
            
        opt.zero_grad()
        costs, _, _ = monte_carlo(policy, h0=h0, trials=batch_size, seed=epoch, verbose=False)
        costs.mean().backward()
        
        with torch.no_grad():
            test_costs, _, _ = monte_carlo(policy, h0=h0, trials=batch_size, seed=TEST, verbose=False)
            test_utils.append(-test_costs.mean())
            print('epoch {}: train utility ({}) test utility ({})'.format(
                epoch, -costs.mean(), test_utils[-1]))
            if epoch % print_every == 0:
                print('\tmu.grad.norm ', policy.mu.grad.norm().item())
                print('\tgamma.grad.norm ', policy.gamma_sqrt.grad.norm().item())
                print('\tcov_sqrt.grad.norm ', policy.cov_sqrt.grad.norm().item())
                print('\tgamma ', policy.gamma_sqrt.detach().item()**2)
                plot_mean_std(policy.mu.detach().numpy(),
                             (policy.cov_sqrt.T @ policy.cov_sqrt).detach().numpy())
                plt.show()
                
                plot_states(policy, h0=h0, seed=TEST, style=STYLE, legend=True)
                plt.show()
                
                plot_controls(policy, h0=h0, seed=TEST, style=STYLE)
                plt.show()
                
        opt.step()
        with torch.no_grad():                                                   
            policy.project()                                                    
            
    return test_utils

In [ ]:
TOTAL_SIMS = 4000
BATCH_SIZE = 10
EPOCHS = int(TOTAL_SIMS / BATCH_SIZE)
LR = 1e-3
PRINT_EVERY = 100 / BATCH_SIZE
MU_INIT = MU
GAMMA_INIT = GAMMA
COV_INIT = COV_SQRT
tuned = Tuner(mu=MU_INIT, gamma=GAMMA_INIT, cov_sqrt=COV_INIT)
test_utils = train(tuned, lr=LR, epochs=EPOCHS, batch_size=BATCH_SIZE, print_every=PRINT_EVERY)

In [ ]:
tuned_h0 = init_holdings(tuned.cov_sqrt.detach().numpy(),
                         tuned.mu.detach().numpy(),
                         tuned.gamma_sqrt.detach().item()**2)
tuned_h0 = torch.tensor(tuned_h0)

In [ ]:
latexify(fig_width=8.0, fig_height=4.0)
fig, axs = plt.subplots(2, 2)

plot_states(untuned, ax=axs[0][0], h0=H0, seed=TEST, xlabel=False, legend=False, style=STYLE)
plot_states(tuned, ax=axs[0][1], h0=tuned_h0, seed=TEST, ylabel=False, xlabel=False, legend=False, style=STYLE)


plot_controls(untuned, ax=axs[1][0], h0=H0, seed=TEST, style=STYLE)
plot_controls(tuned, ax=axs[1][1], h0=tuned_h0, seed=TEST, ylabel=False,style=STYLE)

plt.savefig('mark-state-ctrl.pdf')

In [ ]:
latexify(fig_width=6.5, fig_height=1.9)

_, ax = plt.subplots(1, 1)
ax.plot([-u for u in test_utils], color='black')
ax.set_ylabel('cost')
ax.set_xlabel('iteration')
ax.set_yticks([-0.00425, -0.00475, -0.00525])
plt.tight_layout()
plt.savefig('mark-training.pdf')

In [ ]:
print(ETFS)


print(tuned.mu.detach().numpy())

In [ ]:
print(MU)

In [ ]:
tuned.gamma_sqrt.item()**2

In [ ]:
est_cov = (tuned.cov_sqrt.T @ tuned.cov_sqrt).detach().numpy()
np.median(np.abs((COV - est_cov) / COV))

In [ ]:
latexify(fig_width=6, fig_height=4)

plt.imshow(est_cov)
plt.colorbar()
plt.xticks(np.arange(N_ASSETS), ETFS, rotation=90)
plt.yticks(np.arange(N_ASSETS), ETFS)
plt.show()